# Attempt to fine-tune GPT3 to translate between languages
The first language will be our made up language (see notes on get_bible as to why we think GPT is cheating)
The thesis is that fine-tuning will cause GPT3 to learn the fake language as a new language by assigning the embeddings of the
new words essentially to the same dimensional space as the english.  Should be trivial to translate it back as it is a word 
for word translation.



In [2]:
from lib.config import get_config
import pandas as pd

bible = pd.read_csv('data/bible.bbe.csv')
bible.dropna(inplace=True)



In [23]:
# We choose eng-web because it is somewhat simple English
# BBE is our goal to translate into because it is also plain English
# But it can't do word to word translation
# NOTE: never ask it to translate for the BBE for training

#SOURCE_VERSIONS = ['eng-web', 'eng-asv', 'eng-kjv2006']
SOURCE_VERSIONS = ['eng-web']
DESTINATION_VERSION = 'engBBE'
LANGUAGE_NAME = 'Birrig'

#PROMPT_PREFIX = "Translate the Bible verse $verse from $source to $desination. The verse is: "
PROMPT_PREFIX = ""
PROMPT_SUFFIX = "\n"
COMPLETION_SUFFIX = '\n###'

EXPERIMENT_NAME = 'divinci'

MODEL_NAME = 'davinci'

def create_prompt(reference, text, source):
    return PROMPT_PREFIX.replace('$verse', reference).replace('$source', source).replace('$desination', LANGUAGE_NAME) + text + PROMPT_SUFFIX


In [3]:
# It performs better when we use the full name instead of uncommon abbreviations
def rename_book(reference):
    book, remainder = reference.split()
    if book == 'MAT':
        book = 'Matthew'
    elif book == 'LUK':
        book = 'Luke'
    elif book == 'JHN':
        book = 'John'
    elif book == 'MRK':
        book = 'Mark'
    return " ".join([book, remainder])

bible['reference'] = bible['0'].apply(rename_book)

In [4]:
# Assuming the gospels have a lot of overlap include the synoptic gospels plus John to train
# then predict Mark (which should be easy as Matthew and Luke may have copied from him)
train = bible[bible['book'].isin(['MAT','LUK','JHN'])]
validate = bible[bible['book'].isin(['MRK'])]
train

,0,book,chapter,verse,eng-web,eng-asv,eng-kjv2006,engBBE,engWycliffe,birrig,reference
23213,MAT 1:1,MAT,1,1,"The book of the genealogy of Jesus Christ, the...","The book of the generation of Jesus Christ, th...","The book of the generation of Jesus Christ, th...","The book of the generations of Jesus Christ, t...","The book of the generacioun of Jhesu Crist, th...","Lxi fuuc us lxi girivelourz us Yizaz Kxvozl, l...",Matthew 1:1
23214,MAT 1:2,MAT,1,2,Abraham became the father of Isaac. Isaac beca...,Abraham begat Isaac; and Isaac begat Jacob; an...,Abraham begat Isaac; and Isaac begat Jacob; an...,The son of Abraham was Isaac; and the son of I...,Abraham bigat Isaac. Isaac bigat Jacob. Jacob ...,Lxi zur us Efvexen hez Ozeek; erw lxi zur us O...,Matthew 1:2
23215,MAT 1:3,MAT,1,3,Judah became the father of Perez and Zerah by ...,and Judah begat Perez and Zerah of Tamar; and ...,And Judas begat Phares and Zara of Thamar; and...,And the sons of Judah were Perez and Zerah by ...,"Judas bigat Fares and Zaram, of Tamar. Fares b...",Erw lxi zurz us Yawex hivi Qivid erw Divex fj ...,Matthew 1:3
23216,MAT 1:4,MAT,1,4,Ram became the father of Amminadab. Amminadab ...,and Ram begat Amminadab; and Amminadab begat N...,And Aram begat Aminadab; and Aminadab begat Na...,And the son of Ram was Amminadab; and the son ...,Esrom bigat Aram. Aram bigat Amynadab. Amynada...,Erw lxi zur us Ven hez Ennorewef; erw lxi zur ...,Matthew 1:4
23217,MAT 1:5,MAT,1,5,Salmon became the father of Boaz by Rahab. Boa...,and Salmon begat Boaz of Rahab; and Boaz begat...,And Salmon begat Booz of Rachab; and Booz bega...,And the son of Salmon by Rahab was Boaz; and t...,"Salmon bigat Booz, of Raab. Booz bigat Obeth, ...",Erw lxi zur us Zepnur fj Vexef hez Fued; erw l...,Matthew 1:5
...,...,...,...,...,...,...,...,...,...,...,...
26987,JHN 21:21,JHN,21,21,"Peter, seeing him, said to Jesus, “Lord, what ...","Peter therefore seeing him saith to Jesus, Lor...","Peter seeing him saith to Jesus, Lord, and wha...","Seeing him, Peter said to Jesus, What about th...","Therfor whanne Petre hadde seyn this, he seith...","Ziiorg xon, Qiliv zeow lu Yizaz, Hxel efual lx...",John 21:21
26988,JHN 21:22,JHN,21,22,"Jesus said to him, “If I desire that he stay u...","Jesus saith unto him, If I will that he tarry ...","Jesus saith unto him, If I will that he tarry ...","Jesus said to him, If it is my desire for him ...","Jhesus seith to him, So I wole that he dwelle ...","Yizaz zeow lu xon, Os ol oz nj wizovi suv xon ...",John 21:22
26989,JHN 21:23,JHN,21,23,This saying therefore went out among the broth...,This saying therefore went forth among the bre...,Then went this saying abroad among the brethre...,So this saying went about among the brothers t...,Therfor this word wente out among the brithere...,Zu lxoz zejorg hirl efual enurg lxi fvulxivz l...,John 21:23
26990,JHN 21:24,JHN,21,24,This is the disciple who testifies about these...,This is the disciple that beareth witness of t...,This is the disciple which testifieth of these...,This is the disciple who gives witness about t...,"This is thilke disciple, that berith witnessyn...",Lxoz oz lxi wozkoqpi hxu gomiz holrizz efual l...,John 21:24


In [19]:
df2 = pd.DataFrame()
for item in SOURCE_VERSIONS:
    df2 = df2.append(pd.DataFrame({
        'prompt': train.apply(lambda x: create_prompt(x['reference'], x[item], item), axis=1),
        'completion': train['birrig'].apply(lambda x: ' ' + x + COMPLETION_SUFFIX)
    }))

# removal all null rows
df2 = df2.dropna()
df2.to_json(f'data/train_{EXPERIMENT_NAME}.jsonl', orient='records', lines=True)
df2


/var/folders/tq/94d0g6xx05x_h_z4rfdfh6bh0000gn/T/ipykernel_80075/2467590727.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame({


,prompt,completion
23213,"The book of the genealogy of Jesus Christ, the...","Lxi fuuc us lxi girivelourz us Yizaz Kxvozl, ..."
23214,Abraham became the father of Isaac. Isaac beca...,Lxi zur us Efvexen hez Ozeek; erw lxi zur us ...
23215,Judah became the father of Perez and Zerah by ...,Erw lxi zurz us Yawex hivi Qivid erw Divex fj...
23216,Ram became the father of Amminadab. Amminadab ...,Erw lxi zur us Ven hez Ennorewef; erw lxi zur...
23217,Salmon became the father of Boaz by Rahab. Boa...,Erw lxi zur us Zepnur fj Vexef hez Fued; erw ...
...,...,...
26987,"Peter, seeing him, said to Jesus, “Lord, what ...","Ziiorg xon, Qiliv zeow lu Yizaz, Hxel efual l..."
26988,"Jesus said to him, “If I desire that he stay u...","Yizaz zeow lu xon, Os ol oz nj wizovi suv xon..."
26989,This saying therefore went out among the broth...,Zu lxoz zejorg hirl efual enurg lxi fvulxivz ...
26990,This is the disciple who testifies about these...,Lxoz oz lxi wozkoqpi hxu gomiz holrizz efual ...


In [16]:
# set environment variable in ipython notebook
import os
os.environ["OPENAI_API_KEY"] = get_config('openai')['api_key']


In [20]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py tools fine_tunes.prepare_data -f data/train_{EXPERIMENT_NAME}.jsonl -q

Analyzing...

- Your file contains 3095 prompt-completion pairs
- There are 1 duplicated prompt-completion sets. These are rows: [1392]
- All prompts end with suffix `\n`
- All completions end with suffix `\n###`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 1 duplicate rows [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `data/train_divinci_prepared (1).jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "data/train_divinci_prepared (1).jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `\n` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=["\n###"]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 2.05 hours to train a `curie` model, and less for `ada` and `babbage

In [22]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.create -t "data/train_{EXPERIMENT_NAME}.jsonl" -m {MODEL_NAME} --no_check_if_files_exist

Upload progress: 100%|███████████████████████| 842k/842k [00:00<00:00, 722Mit/s]
Uploaded file from data/train_divinci.jsonl: file-2GuJZRLN25fQyUIeJQFXxGXg
Created fine-tune: ft-OrA8XcaqR2jreSMBfBo9mDx9
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-04-01 18:33:07] Created fine-tune: ft-OrA8XcaqR2jreSMBfBo9mDx9
[2023-04-01 18:34:04] Fine-tune costs $33.88
[2023-04-01 18:34:05] Fine-tune enqueued. Queue number: 0
[2023-04-01 18:34:06] Fine-tune started

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-OrA8XcaqR2jreSMBfBo9mDx9



In [26]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.follow -i ft-OrA8XcaqR2jreSMBfBo9mDx9

[2023-04-01 18:33:07] Created fine-tune: ft-OrA8XcaqR2jreSMBfBo9mDx9
[2023-04-01 18:34:04] Fine-tune costs $33.88
[2023-04-01 18:34:05] Fine-tune enqueued. Queue number: 0
[2023-04-01 18:34:06] Fine-tune started
[2023-04-01 18:49:07] Completed epoch 1/4
[2023-04-01 19:15:30] Completed epoch 3/4
[2023-04-01 19:29:20] Uploaded model: davinci:ft-personal-2023-04-02-02-29-19
[2023-04-01 19:29:21] Uploaded result file: file-uUOesh3TaDTlOjVICgzmTmGF
[2023-04-01 19:29:21] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m davinci:ft-personal-2023-04-02-02-29-19 -p <YOUR_PROMPT>


In [27]:
import openai
from lib.cipher import substitution_cipher
from nltk.translate.bleu_score import sentence_bleu

openai.api_key = os.getenv("OPENAI_API_KEY")

#TODO: Test with lower temperature
def translate(text, debug=False, temperature=0.5):
    """
    Translate text from English to Birrig
    
    Parameters
    ----------
    text: str|array
        The text to translate. If an array is passed, each element will be translated.

    Returns
    ------- 
    array
        An array of translations
        
    """
    if isinstance(text, list):
        text = [row + PROMPT_SUFFIX for row in text]
        if len(text) > 20:
            # break it into chunks of 20 and call translate on each chunk
            # then combine the results
            result = []
            for i in range(0, len(text), 20):
                result += translate(text[i:i+20])
            return result

    else:
        text += PROMPT_SUFFIX

    response = openai.Completion.create(
        model="davinci:ft-personal-2023-04-02-02-29-19",
        prompt=text,
        temperature=temperature,  # 0.1 is very little randomness/creativity, 2 is very
        max_tokens=100,
        n=1,
        logprobs=debug and 5 or 0,
        frequency_penalty=0,
        presence_penalty=0,
        stop=[COMPLETION_SUFFIX]
    )
    result = []
    for choice in response.get('choices',[{}]):
        decoded = substitution_cipher(choice.get('text').strip(), encode=False).strip()
        result.append(decoded)
        if debug:
            # convert logprobs to probabilities
            logprobs = choice.get('logprobs', {}).get('token_logprobs', [])
            tokens = [substitution_cipher(x, encode=False) for x in choice.get('logprobs', {}).get('tokens', [])]
            probs = [10**logprob for logprob in logprobs]
            # merge the probabilities with the tokens
            probs = list(zip(tokens, probs))

            print("PROBS: ", probs)
            print("TRANSLATION: ", choice.get('text').strip())
            print("DECODED: ", decoded)
            # result.append({
        #     'translation': choice.get('text').strip(),
        #     'decoded_translation': decoded,
        #     'bleu_score': sentence_bleu([text.split()], decoded.split())
        # })
    
    return result

In [28]:
bible[bible['0']=='MRK 1:6'][['eng-web']].values[0][0]

'John was clothed with camel’s hair and a leather belt around his waist. He ate locusts and wild honey.'

In [29]:
# Get the column eng-web from the first 3 rows of validate then send it to translate
validate['translation'] = translate(validate['eng-web'].to_list())
validate

/var/folders/tq/94d0g6xx05x_h_z4rfdfh6bh0000gn/T/ipykernel_80075/591098392.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validate['translation'] = translate(validate['eng-web'].to_list())


,0,book,chapter,verse,eng-web,eng-asv,eng-kjv2006,engBBE,engWycliffe,birrig,reference,translation
24284,MRK 1:1,MRK,1,1,The beginning of the Good News of Jesus Christ...,"The beginning of the gospel of Jesus Christ, t...","The beginning of the gospel of Jesus Christ, t...",The first words of the good news of Jesus Chri...,"The bigynnyng of the gospel of Jhesu Crist, th...",Lxi sovzl huvwz us lxi guuw rihz us Yizaz Kxvo...,Mark 1:1,"The first of the good news of Jesus Christ, th..."
24285,MRK 1:2,MRK,1,2,"As it is written in the prophets, “Behold, I s...","Even as it is written in Isaiah the prophet, B...","As it is written in the prophets, Behold, I se...",Even as it is said in the book of Isaiah the p...,"As it is writun in Ysaie, the prophete, Lo! Y ...",Imir ez ol oz zeow or lxi fuuc us Ozeoex lxi q...,Mark 1:2,"And as it is in the prophets, There is a reque..."
24286,MRK 1:3,MRK,1,3,"the voice of one crying in the wilderness, ‘Ma...","The voice of one crying in the wilderness, Mak...","The voice of one crying in the wilderness, Pre...","The voice of one crying in the waste land, Mak...","The vois of a crier in desert, Make ye redi th...","Lxi muoki us uri kvjorg or lxi hezli perw, Nec...",Mark 1:3,"And the voice of one crying in the waste land,..."
24287,MRK 1:4,MRK,1,4,John came baptizing in the wilderness and prea...,"John came, who baptized in the wilderness and ...","John did baptize in the wilderness, and preach...","John came, and gave baptism in the waste land,...","Joon was in desert baptisynge, and prechynge t...","Yuxr keni, erw gemi feqlozn or lxi hezli perw,...",Mark 1:4,And John came to the giving baptism by giving ...
24288,MRK 1:5,MRK,1,5,All the country of Judea and all those of Jeru...,And there went out unto him all the country of...,And there went out unto him all the land of Ju...,And there went out to him all the people of Ju...,"And al the cuntre of Judee wente out to hym, a...",Erw lxivi hirl ual lu xon epp lxi qiuqpi us Ya...,Mark 1:5,And all the country of Judaea and all those of...
...,...,...,...,...,...,...,...,...,...,...,...,...
24957,MRK 16:16,MRK,16,16,He who believes and is baptized will be saved;...,He that believeth and is baptized shall be sav...,He that believeth and is baptized shall be sav...,He who has faith and is given baptism will get...,"Who that bileueth, and is baptisid, schal be s...",Xi hxu xez seolx erw oz gomir feqlozn hopp gil...,Mark 16:16,"He who has faith and beested by grace, will ha..."
24958,MRK 16:17,MRK,16,17,These signs will accompany those who believe: ...,And these signs shall accompany them that beli...,And these signs shall follow them that believe...,And these signs will be with those who have fa...,"And these tokenes schulen sue hem, that bileue...",Erw lxizi zogrz hopp fi holx lxuzi hxu xemi se...,Mark 16:17,These signs will be with those who have faith:...
24959,MRK 16:18,MRK,16,18,they will take up serpents; and if they drink ...,"they shall take up serpents, and if they drink...",They shall take up serpents; and if they drink...,"They will take up snakes, and if there is pois...",thei schulen do awei serpentis; and if thei dr...,"Lxij hopp leci aq zreciz, erw os lxivi oz quoz...",Mark 16:18,"And they will take up spirits, and if they tak..."
24960,MRK 16:19,MRK,16,19,"So then the Lord, after he had spoken to them,...","So then the Lord Jesus, after he had spoken un...","¶ So then after the Lord had spoken unto them,...","So then the Lord Jesus, after he had said thes...","And the Lord Jhesu, aftir he hadde spokun to h...","Zu lxir lxi Puvw Yizaz, esliv xi xew zeow lxiz...",Mark 16:19,"Then the Lord, hearing to them, went up in hea..."


In [30]:
# drop rows that have a null value in any column
validate = validate.dropna()
validate['bleu_score'] = validate.apply(lambda row: sentence_bleu([row['eng-web'].split(), row['eng-asv'].split(), row['eng-kjv2006'].split(),row['engBBE'].split()], row['translation'].split()), axis=1)

/Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

In [75]:
# describe the bleu score column
validate['bleu_score'].describe()

count     6.730000e+02
mean      3.856940e-01
std       2.568575e-01
min      1.234056e-231
25%       2.204887e-01
50%       3.828752e-01
75%       5.531961e-01
max       1.000000e+00
Name: bleu_score, dtype: float64

## Analysis
This theory proposes that to avoid word for word translation start with the web english version and target the BBE english version.  That way it has to use alternate words to find the way.

It generally does okay and makes mostly readable text.  Some weaknesses observed are
 - might get distracted and start repeating itself
 - likes to start most verses with "And"

| Stat | Number
| ----- | ----- |
| Average bleu score | 0.387 |
| 75 percentile | 0.553 |

In [31]:
# foreach row in validate, print the column "0" and translation
for index, row in validate.iterrows():
    print(row['0'], row['translation'])

MRK 1:1 The first of the good news of Jesus Christ, the Son of God.
MRK 1:2 And as it is in the prophets, There is a request for my servant to be sent before your face, who will make ready your way before you:
MRK 1:3 And the voice of one crying in the waste land, Make ready the way of the Lord; make his ways straight.
MRK 1:4 And John came to the giving baptism by giving baptism for all those who were dead in the sins of the forgiveness of sins.
MRK 1:5 And all the country of Judaea and all those of Jerusalem went out to him; and he was teaching them in the Jordan, saying what evil sin they had done.
MRK 1:6 And John was cleaned with camel's heel, and a leaven garded near his waste, and he took food of camels and wine.
MRK 1:7 And he gave knowledge of the coming of him, saying, Then the man who is greater than me comes, whose shoes I am not good enough to take them from his heads and make them dry.
MRK 1:8 Then I gave you baptism in water, but he will give you baptism in the Holy Spir